In [2]:
%load_ext autoreload
%autoreload 2

In [3]:

# coding: utf-8
import requests
import json
import urllib.parse
import logging
import logging.config
import configparser

import pandas as pd
import numpy as np
os.environ['INIDIR'] = '/Users/macico/Dropbox/kaggle/kuloko/ini'
os.environ['LOGDIR'] = '/Users/macico/Dropbox/kaggle/kuloko/log'
INIDIR=os.environ['INIDIR'] 
LOGDIR=os.environ['LOGDIR'] 

In [4]:


sys.path.append(os.path.join(os.path.dirname('__file__'),'..'))
import  kuloko_handler.handler.socket_handler as skt_api
private_api_ini = configparser.ConfigParser()
private_api_ini.read(os.path.join(INIDIR,'private_api.ini'), encoding='utf-8')

general_config_ini = configparser.ConfigParser()
general_config_ini.read(os.path.join(INIDIR,'config.ini'), encoding='utf-8')

logging.config.fileConfig(os.path.join(INIDIR,'logconfig.ini'),defaults={'logfilename': os.path.join(LOGDIR,'logging.log')})
logger = logging.getLogger("KULOKO")

In [5]:
from kuloko_db.src.mongo_handler import MongoHandler
mongo_ini = configparser.ConfigParser()
mongo_ini.read(os.path.join(INIDIR,'db_config.ini'), encoding='utf-8')
mongo_db = MongoHandler(mongo_ini['LOCAL'],"orderbook")

In [5]:

test = skt_api.Trade("token",logger, general_config_ini,private_api_ini)

TypeError: __init__() takes 4 positional arguments but 5 were given

In [5]:
test.get_access_token()

NameError: name 'test' is not defined

In [6]:
test.extend_access_token()

NameError: name 'test' is not defined

(json.dumps(a.json(), indent=2))

In [6]:
trade = skt_api.Trade(logger, general_config_ini,private_api_ini)

In [7]:
trade.connect(trade.get_url(),'BTC')

In [8]:
trade.subscribe()


In [14]:
a = trade.get()

In [16]:
trade.convert_shape(a,'json')

[{'channel': 'trades',
  'price': 2049998.0,
  'side': 'SELL',
  'size': 0.1,
  'symbol': 'BTC',
  'timestamp': '20201130055238280000'},
 {'channel': 'trades',
  'price': 2049998.0,
  'side': 'BUY',
  'size': 0.1,
  'symbol': 'BTC',
  'timestamp': '20201130055238280000'}]

In [200]:
# str_utc_to_dt_offset(a[0]["timestamp"],trade.tz_offset)

In [9]:
trade.disconnect()

In [10]:
orderbooks = skt_api.Orderbooks(logger, general_config_ini,private_api_ini)

In [11]:
orderbooks.connect(orderbooks.get_url(),'BTC')

In [12]:
orderbooks.subscribe()

In [14]:
data = orderbooks.get()

In [16]:
# insert_json =[ orderbooks.convert_shape(_data,5,"json") for _data in data ]
# mongo_db.insert_many(insert_json)
for db_data in mongo_db.db['orderbook'].find():
    print(db_data)

{'_id': ObjectId('5fc957619464573810be8871'), 'time': '20201203122343933000', 'symbol': 'BTC', 'bids0': 2015000.0, 'bids0_size': 0.0052, 'bids1': 2014520.0, 'bids1_size': 0.1, 'bids2': 2014511.0, 'bids2_size': 0.4, 'bids3': 2014510.0, 'bids3_size': 0.1, 'bids4': 2014265.0, 'bids4_size': 0.2, 'asks0': 2015000.0, 'asks0_size': 0.0052, 'asks1': 2014520.0, 'asks1_size': 0.1, 'asks2': 2014511.0, 'asks2_size': 0.4, 'asks3': 2014510.0, 'asks3_size': 0.1, 'asks4': 2014265.0, 'asks4_size': 0.2}
{'_id': ObjectId('5fc957619464573810be8872'), 'time': '20201203122344182000', 'symbol': 'BTC', 'bids0': 2015000.0, 'bids0_size': 0.0052, 'bids1': 2014511.0, 'bids1_size': 0.4, 'bids2': 2014510.0, 'bids2_size': 0.1, 'bids3': 2014270.0, 'bids3_size': 0.1, 'bids4': 2014265.0, 'bids4_size': 0.2, 'asks0': 2015000.0, 'asks0_size': 0.0052, 'asks1': 2014511.0, 'asks1_size': 0.4, 'asks2': 2014510.0, 'asks2_size': 0.1, 'asks3': 2014270.0, 'asks3_size': 0.1, 'asks4': 2014265.0, 'asks4_size': 0.2}
{'_id': ObjectId('

In [24]:
mongo_db.delete_all()

In [13]:
orderbooks.on_close()

In [14]:
orderbooks.disconnect()

In [48]:
ticker = skt_api.Ticker(logger, general_config_ini,private_api_ini)

In [49]:
ticker.connect(ticker.get_url(),'BTC')

In [50]:
ticker.subscribe()

In [51]:
data = ticker.get()

In [60]:
ticker.disconnect()

In [56]:
ticker.convert_shape(data[0],'json')

{'ask': 2003867.0,
 'bid': 2003866.0,
 'high': 2020250.0,
 'last': 2003867.0,
 'low': 1916900.0,
 'volume': 634.1223,
 'timestamp': '20201202044346421000',
 'symbol': 'BTC'}